# PBSHM Research Template
This Jupyter Notebook template has been put together to support the research of PBSHM without the need of developing a full PBSHM Framework module. The commands to connect to the database and translate timestamp have the same names that are present within the PBSHM Framework, as such, do not change these names as it will delay the adoption of your code within the framework. Feel free to remove/edit this markdown block as appropriate for your Jupyter Notebook when you publish to your own repo, however, the following code block will need to remain.

In [ ]:
# Import packages
import pymongo
from json import load as load_json
from urllib.parse import quote_plus
from datetime import datetime
from pytz import utc

# Simulate PBSHM methods
def db_connect():
    config = load_json(open("database.json"))
    mongodb_uri = f"mongodb://{quote_plus(config['authentication']['username'])}:{quote_plus(config['authentication']['password'])}@{config['hostname']}:{config['port']}/{config['authentication']['database']}"
    return pymongo.MongoClient(mongodb_uri)[config['database']]

def structure_collection():
    return db_connect()["structures"]

def nanoseconds_since_epoch_to_datetime(nanoseconds):
    return datetime.fromtimestamp(int(nanoseconds * 0.000000001), utc)

## Example
One example of how you can use the above code to connect to the PBSHM database and retrieve information on a population. For the example, I am using an [aggregation pipeline](https://www.mongodb.com/docs/manual/core/aggregation-pipeline/) to load any documents which have "documentation-example" set as their population and then limiting the results to the first result only. This markdown block and the code block below can be removed on your own version. 


In [ ]:
for document in structure_collection().aggregate([
    {"$match": {"population": {"$eq": "documentation-example"}}},
    {"$limit": 1}
]):
    print(f"Structure Name: {document['name']}")
    print(f"Timestamp: {nanoseconds_since_epoch_to_datetime(document['timestamp'])}")
    for channel in document["channels"]:
        print(f"\tChannel: {channel['name']} Type: {channel['type']} Unit: {channel['unit'] if 'unit' in channel else None} Value: {channel['value']}")